# Atividade de Laboratório 1
### Members
- Leonardo Mallmann   21104259
- Caetano Muller      21109037
- Euzébio Hensel      


### PLEASE RUN EACH CELL AT A TIME NOT ALTOGETHER!!!!!!!!!

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.error import URLError, HTTPError, ContentTooShortError
import re
import csv
import json

In [ ]:
def download(url, user_agent='wswp', charset='utf-8'):
  #print('Downloading:', url)
  request = urllib.request.Request(url)
  request.add_header('User-agent', user_agent)
  try:
    resp = urllib.request.urlopen(request)
    cs = resp.headers.get_content_charset()
    if not cs:
        cs = charset
    html = resp.read().decode(cs)
  except (URLError, HTTPError, ContentTooShortError) as e:
    print('Download error:', e.reason)
  return html

# 1

The following cell runs the code which collects the data from the countries webtsite

In [ ]:
def scrap(html_content):
  soup = BeautifulSoup(html_content, 'html5lib')
  rows = soup.select('table tbody tr')
  tiempo = datetime.now()
  field_names = []
  field_values = []
  for row in rows:
    c = row.select('.w2p_fl label');
    field_names.append(c[0].string.strip().replace(':', ''));
    v = row.select('.w2p_fw');
    if not v[0].string:
      if len(re.findall('<img', str(v[0]))) != 0:
        img_src = v[0].select('img')[0];
        field_values.append(img_src['src'])
        continue
      if len(re.findall('<div', str(v[0]))) != 0:
        a_links = v[0].select('div a');
        neighbours = [a_tag.string.strip() for a_tag in a_links]
        field_values.append(neighbours);
        continue
      field_values.append('None')
      continue
    if v[0].string == ' ':
      field_values.append('None')
      continue
    field_values.append(v[0].string)
  field_names.append('Date/Time')
  field_values.append(tiempo)
  
  return field_values, field_names
    

### CSV DATA INSERTION


In [ ]:
def insert_headers(fieldnames):
  with open('places.csv', 'w', newline='\n') as places_csv:
    writer = csv.DictWriter(places_csv, fieldnames=fieldnames)
    writer.writeheader()

def insert_data(data_row, fieldnames):
  with open('places.csv', 'a', newline='\n') as places_csv:
    writer = csv.DictWriter(places_csv, fieldnames=fieldnames)
    
    writer.writerow({
      f_name: f_value
      for f_name, f_value in zip(fieldnames,data_row)
    }) # or dict(zip(keys, values))

In [ ]:
def crawl_sitemap(url):
  sitemap = download(url)
  links = re.findall('\d(\/.*)<\/loc>', sitemap)
  for index, link in enumerate(links):
    html = download(f'http://localhost:8000{link}')
    f_values, f_names = scrap(html)
    if index==0:
      insert_headers(f_names)
    insert_data(f_values, f_names)

### REPLACE THE URL IN crawl_sitemap

In [ ]:
crawl_sitemap('http://localhost:8000/places/static/sitemap.xml');

In [ ]:
def page_monitor(url, csv_path):
    sitemap = download(url)
    lcsv = list(csv.reader(open(csv_path)))
    links = re.findall('\d(\/.*)<\/loc>', sitemap)
    page_number = 1;
    for index, link in enumerate(links):
        html = download(f'http://localhost:8000{link}')
        f_values, f_names = scrap(html)
        for j, value in enumerate(f_values):
            if j!=len(f_values)-1:
                if lcsv[page_number][j] != f_values[j]:
                    if isinstance(f_values[j], list):
                      if lcsv[page_number][j] != str(f_values[j]):
                        lcsv[page_number][j] = f_values[j]
                        lcsv[page_number][len(f_values)-1] = datetime.now()
                        continue
                      continue
                    lcsv[page_number][j] = f_values[j]
                    lcsv[page_number][len(f_values)-1] = datetime.now()
        if index==2:
            break
        page_number+=1
    writer = csv.writer(open(csv_path, 'w'))
    writer.writerows(lcsv)
        
    
page_monitor('http://localhost:8000/places/static/sitemap.xml', 'places.csv')

## 2
# How to execute:

Literally just run the import cell and the ones mentioned previously

this one consists of basically our methods of scrapping the content from the website
with the function imdb_scrap() being called by run_imdb(), this one will call the cJSON()
function givimg the dictionary list collected as a parameter to then and only then
create the JSON file

the imdb top 250 movies website can be found [here](https://www.imdb.com/chart/top/)

In [21]:
def cJSON(dl):
  with open('top_250_movies.json', 'w') as f:
    json.dump(dl, f)

  return 0
  

def imdb_scrap(html_content):
  soup = BeautifulSoup(html_content, 'html5lib')
  rows = soup.select('table tbody tr')
  dl = []
  count=0

  for index,row in enumerate(rows):
    count+=1
    #getting the posters class
    poster_a = row.select('.posterColumn a')
    #selecting image class
    img=poster_a[0].select('img')
    #getting both the redirection url and img source
    poster_url = poster_a[0]['href']
    poster_img = img[0]['src']
    #class of the title year and such...
    title_class = row.select('.titleColumn a')
    movie_title = title_class[0].string
    year_class = row.select('.titleColumn span')
    movie_year = year_class[0].string.replace('(', '').replace(')', '')
    rating_class = row.select('.ratingColumn.imdbRating strong')
    movie_rating = rating_class[0].string
    #specific_page_path = row.select('.titleColumn a')[0]['href']
    specific_page_url = f'https://www.imdb.com{poster_url}'
    specific_page_html = download(specific_page_url)
    selector = BeautifulSoup(specific_page_html, 'html5lib')
    movie_directors_tag = selector.select('.sc-fa02f843-0.fjLeDR > ul > li:first-child a')
    movie_directors_arr = [director.string for director in movie_directors_tag]
    genders_tag = selector.select('.ipc-chip-list.sc-16ede01-4.bMBIRz span')
    genders_arr = [gender.string for gender in genders_tag]
    popularity_tag = selector.select('.sc-edc76a2-2.geydkP')
    popularity = popularity_tag[0].string if popularity_tag else 'None'
    field_values = [movie_title, movie_year, poster_url, poster_img, movie_rating, popularity,
        movie_directors_arr, genders_arr]
    field_names = ['title', 'year', 'poster_url',
        'poster_img', 'imdb_rating', 'popularity', 'directors', 'genders']
    d = dict(zip(field_names, field_values))
    dl.append(d)
  return cJSON(dl)

def run_imdb(url):
  a = download(url)
  return imdb_scrap(a)

run_imdb('https://www.imdb.com/chart/top/')


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


0